# ESRGAN

This notebook is a simple implementation of the SRGAN super resolution architecture. Further Improvements need to be done. 

### Key takeaways from the paper

- Ability of Mean Squared Error, Peak Signal to Noise Ratio to capture 'perceptually relevant'(like texture for instance) differences like high texture details is limited as both are based on pixel wise image differences. 

- Super resolved image may not be as photo realistic as the original image. 

- A Super Resolution Generative Adversarial Network is proposed, in which a deep residual network(ResNet) with skip connections. 

- A novel loss function is also proposed which uses high level feature maps of the VGG network. 

- Low resolution images were obtained by applying a Gaussian filter to the High Resolution Image, proceeded by a downsampling with a factor of r. 

- The ParametricReLU is used as the activation function. 

- The perceptual loss function was defined as a sum of content loss and adversarial loss. Weightage of the loss function is $1 : \frac{1}{1000}$(content loss : adversarial loss)

- Perceptual Loss > can use many humans to eval images, a far practical method is to use a pre trained network that has been trained on millions of images.

#### Things to read up on:
- MSE, PSNR 
- ResNet, skip connections
- Loss functions
- feature maps, vgg network

## Dependencies

In [1]:
import torch
import math
from os import listdir
import numpy as np
from torch.autograd import Variable
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from os.path import join
from torch import nn, optim
from torchvision.models.vgg import vgg16
from tqdm import tqdm
import os
from time import time
torch.autograd.set_detect_anomaly(True)

In [2]:
import sys
from math import log10, sqrt 
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import cv2
import torchvision.transforms as transforms
from scipy.ndimage import rotate
from time import time
import warnings as wr
wr.filterwarnings("ignore")
from pylab import rcParams
#rcParams['figure.figsize'] = 9,5

In [3]:
UPSCALE_FACTOR = 4
CROP_SIZE = 88

In [4]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

## Image Metrics

In [5]:
def PSNR(i, j): 
    mse = np.mean((i - j) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 

In [6]:
def SSIM(i,j):
    i = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    j = cv2.cvtColor(j, cv2.COLOR_BGR2GRAY)
    s = ssim(i, j)
    return s

In [7]:
def MSE(i, j):
    # the 'Mean Squared Error' between the two images is the sum of the squared difference between the two images
    mse_error = np.sum((i.astype("float") - j.astype("float")) ** 2)
    mse_error /= float(i.shape[0] * i.shape[1])
    # return the MSE. The lower the error, the more "similar" the two images are.
    return mse_error

## Helper Functions

In [8]:
def readRGBImage(path):
    '''
    A simple function to read images in RGB, as OpenCV reads images in BGR, and can mess up displaying images. 
    
    inputs: 
        path(str): this is the path of the image
    '''
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

In [9]:
def ScaleImageByRatio(image, factor, interpolationMethod = cv2.INTER_LINEAR):
    '''
    Scale the image using a scaling factor
    
    inputs:
        image - input image as a matrix
        factor - scale factor for both length and width 
        interpolationMethod - interpolation method used
    '''
    
    return cv2.resize(image,None,fx=factor, fy=factor, interpolation = interpolationMethod)

In [10]:
def DisplayImageComparison1x3(hr, lr, sr, lr_title = 'LR Image', sr_title = 'SR Image', metrics=False):
    '''
    This function shows a comparason of the ground truth HR images, LR images and the SR images. 
    
    input: 
        hr(np.ndarray): the HR, ground-truth Image
        lr(np.ndarray): the LR Image
        hr(np.ndarray): the SR Image.
    '''
    if metrics:
        sr_title = f'SR Image\nPSNR: {PSNR(sr,hr):.3f}, SSIM: {SSIM(sr,hr):.3f}'
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(11,5))
    ax = axes.ravel()
    ax[0].imshow(hr)
    ax[0].set_title('HR Image')
    ax[1].imshow(lr)
    ax[1].set_title(lr_title)
    ax[2].imshow(sr)
    ax[2].set_title(sr_title)
    #plt.tight_layout()
    plt.show()

In [11]:
def getGPU_details(device):
    '''
    A simple function which prints information about your gpu. 
    '''
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Total Memory: {round(torch.cuda.get_device_properties(0).total_memory/1024**3,1)} GB')
    print('Memory Usage:')
    print(f'Allocated: {round(torch.cuda.memory_allocated(0)/1024**3,1)} GB')
    print(f'Cached: {round(torch.cuda.memory_reserved(0)/1024**3,1)} GB')

## Conventional Methods
![](upsampling_methods.png)

### Nearest Neighbour Interpolation

Nearest Neighbour Interpolation in Images is a simple method used to resize images by determining the intensity value of new pixels based on the nearest existing pixel. Imagine you have a small picture and you want to make it bigger. Nearest Neighbor Interpolation helps decide what color each new pixel should be by looking at the closest original pixels. It's like if you have a small drawing and you want to make it twice as big, you would look at each small square and color the new big squares based on the closest small square's color. This method is quick and easy to understand, making it a common way to resize images.

In [ ]:
i = readRGBImage('source/test_images/baboon.png')
j = readRGBImage('source/test_images/baboon_test.png')
a = ScaleImageByRatio(j, UPSCALE_FACTOR, interpolationMethod=cv2.INTER_NEAREST)
DisplayImageComparison1x3(i, j, a, sr_title='Image Upscaled by\nNearest Neighbour Interpolation')

### Bilinear Interpolation

First, we perform linear interpolation in the x-direction at the y-coordinate of the new pixel. This means we find the values of the two closest pixels along the same row as the new pixel and calculate a weighted average of those two values based on the distance of the new pixel from each of those pixels.

Next, we perform linear interpolation in the y-direction at the x-coordinate of the new pixel. This means we find the values of the two closest pixels along the same column as the new pixel and calculate a weighted average of those two values based on the distance of the new pixel from each of those pixels.

Finally, we take the weighted average of the two interpolated values obtained in the previous steps to get the final value of the new pixel.

In [ ]:
i = readRGBImage('source/test_images/baboon.png')
j = readRGBImage('source/test_images/baboon_test.png')
a = ScaleImageByRatio(j, UPSCALE_FACTOR, interpolationMethod=cv2.INTER_LINEAR)
DisplayImageComparison1x3(i, j, a, sr_title='Image Upscaled by\nBilinear Interpolation')

### Bicubic Interpolation

Bilinear Interpolation uses the 4 closest pixels to estimate the value of a new pixel, whereas Bicubic Interpolation uses the 16 closest pixels. This allows Bicubic Interpolation to produce smoother and more accurate results, especially for images with fine details.

In [ ]:
i = readRGBImage('source/test_images/baboon.png')
j = readRGBImage('source/test_images/baboon_test.png')
a = ScaleImageByRatio(j, UPSCALE_FACTOR, interpolationMethod=cv2.INTER_CUBIC)
DisplayImageComparison1x3(i, j, a, sr_title='Image Upscaled by\nBicubic Interpolation')

# Dataset

This section contains code for the dataloader needed for the data in the folder. This is a custom implementation of the `TrainDatasetFromFolder` class, which inherits from the original class in the `pytorch` library. The dataloader returns a HR(High Resolution, which is the image at its original resolution) image and an LR image(which has been resized to introduce image degradation).

The dataset we used in this notebook is the [Alsat-2B satellite dataset](https://github.com/achrafdjerida/Alsat-2B), which was introduced in the paper, [A New Public Alsat-2b dataset for Single-Image Super-Resolution](https://arxiv.org/pdf/2103.12547.pdf) which contains 2182 training image pairs of HR and LR images. The HR images have dimensions of 256x256 pixels, whereas the LR images have dimensions of 64x64 pixels. The Test dataset is split into 3 categories, each containing scenes from Agriculture, Urban, and a special category. These categories have 56 HR-LR image pairs each. 

In [12]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

In [13]:
def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)

In [14]:
def train_hr_transform(crop_size):
    return Compose([
        RandomCrop(crop_size),
        ToTensor(),
    ])

In [15]:
def train_lr_transform(crop_size, upscale_factor):
    return Compose([
        ToPILImage(),
        Resize(crop_size // upscale_factor, interpolation=Image.BICUBIC),
        ToTensor()
    ])

In [16]:
def display_transform():
    return Compose([
        ToPILImage(),
        Resize(400),
        CenterCrop(400),
        ToTensor()
    ])

In [17]:
class TrainDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, crop_size, upscale_factor):
        super(TrainDatasetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]
        crop_size = calculate_valid_crop_size(crop_size, upscale_factor)
        self.hr_transform = train_hr_transform(crop_size)
        self.lr_transform = train_lr_transform(crop_size, upscale_factor)

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.image_filenames[index]))
        lr_image = self.lr_transform(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_filenames)

In [19]:
train_set = TrainDatasetFromFolder("source/Alsat-2B/Train/HR", crop_size=CROP_SIZE,
                                   upscale_factor=UPSCALE_FACTOR)
trainloader = DataLoader(train_set, batch_size=64, num_workers=0, shuffle=True)

# Model Implementation

![SRGAN Model Architecture](images/esrgan.png)

## Discriminator architecture

The Discriminator block consists of a block which contains a convolutional layer, followed by batch normalisation, followed by an activation layer. The discriminator block then progressess to have the number of filters multiplied by 2(first block has 64 filters, next block has 128 filters, next block has 256 and so on and so forth). In addition, blocks with the same number of filters have alternating stride(2 blocks having 128 filters have strides of 1 for the first block, and 2 for the second block, same can be observed with the 256 filters blocks and the 512 filters blocks). This is then fed into a dense layer with 1024 nodes, which is fed into an activation function(LeakyReLU) and then into another dense layer with 1 node, whose output is fed into a sigmoid activation function. 

## Generator Architecture

The first major block consists of a sequence of residual blocks, after the input layer. 

### What is a Residual Block?

A Residual block consists of the following components:
- A series of convolutional layers that are responsible for extracting features from the input data. 
- Batch normalisation is applied after each convolutional layer. This helps in stabilising and accelerating training by normalising the input to the following layer. 
- Activation functions are applied to introduce non linearity in the network, which allows the network to learn complex and non linear patterns in the data. 
- Skip Connections which are essentially the sum of the input of the block and the output of the last batch normalisation layer, allows gradients to flow more freely throught the network. 

A residual block is a fundamental component of ResNet, a model for extracting features from an image. It is designed to address the degradation problem in deep neural networks, which occurs when the performance of a deep neural network decreases as the number of layers increases

> Note: Read up on the Vanishing Gradient Problem

In [ ]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, up_scale):
        super(UpsampleBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * up_scale ** 2,
                              kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(up_scale)
        self.prelu = nn.PReLU()
    def forward(self, x):
        x = self.conv(x)
        x = self.pixel_shuffle(x)
        x = self.prelu(x)
        return x

## What are Residual Dense Blocks?

In [20]:
class ResidualDenseBlock(nn.Module):
    def __init__(self, inChannels=64, outChannels=32, beta=0.2):
        super().__init__()
        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(inChannels+outChannels, outChannels, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(inChannels + 2 * outChannels, outChannels, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(inChannels + 3 * outChannels, outChannels, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(inChannels + 4 * outChannels, outChannels, kernel_size=3, stride=1, padding=1)
        self.act = nn.LeakyReLU()
        self.beta = beta
    
    def forward(self, x):
        block1 = self.act(self.conv1(x))
        block2 = self.act(self.conv2(torch.cat((block1, x), dim = 1)))
        block3 = self.act(self.conv3(torch.cat((block2, block1, x), dim = 1)))
        block4 = self.act(self.conv4(torch.cat((block3, block2, block1, x), dim = 1)))
        out = self.conv5(torch.cat((block4, block3, block2, block1, x), dim = 1))
        return x + self.beta * out

## What are Residual in Residual Dense Blocks?

In [21]:
class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, inChannels=64, outChannels=32, beta=0.2):
        super().__init__()
        self.RDB = ResidualDenseBlock(inChannels, outChannels)
        self.beta = beta
        
    def forward(self, x):
        out = self.RDB(x)
        out = self.RDB(out)
        out = self.RDB(out)
        return x + self.beta * out

No of Residual-in-Residual Blocks: 23

In [22]:
class Generator(nn.Module):
    def __init__(self, inChannels=3, outChannels=3, n_blocks=23):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        RRDB = ResidualInResidualDenseBlock()
        RRDB_layer = []
        for i in range(n_blocks):
            RRDB_layer.append(RRDB)
        self.RRDB_block = nn.Sequential(*RRDB_layer) # *RRDB_Layer is a packing argument. 
        self.RRDB_conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.upconv = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.output_conv = nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1)
        
    def forward(self, x):
        first_conv = self.conv1(x)
        RRDB_block = torch.add(self.RRDB_conv2(self.RRDB_block(fist_conv)), first_conv)
        upsample_1 = self.upconv(f.interpolate(RRDB_block, scale_factor=2))
        upsample_2 = self.upconv(f.interpolate(upsample_1, scale_factor=2))
        output = self.output_conv(upsample_2)
        return output

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(512, 1024, kernel_size=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(1024, 1, kernel_size=1)
        )
    def forward(self, x):
        batch_size=x.size()[0]
        return torch.sigmoid(self.net(x).view(batch_size))

## Loss Functions

A number of loss functions have been devised over the past years to train SR models. These can be categorised into 2 categories: 

1. Pixel Based Losses
2. Network Based Lossed

#### Pixel Based Losses

- Pixel Loss(MSE): This is Mean Squared Error, in which each pixel of the SR image is compared against the corresponding pixel in the HR image. 
- Mean Absolute Error: This is the sum of the absolute difference between the predicted values of the model and the actual target values. 
- Total Variation Loss(TV): This method is used to supress the noise in the generated image. It measures the noise in the image by taking the absolute difference between neighbouring pixels. 

#### Network Based Losses

- Perceptual Losses: This loss function evaluates based on the perceptual quality of the generated SR image as compared to the perceptual quality of the HR image. The way this is implemented is using the high level features of a pretrained image classification network. 
- Texture Loss: This loss function is used to enable the generated SR image to have the same style as the HR image(can include texture, colour, contrast in the image). 
- Adversarial Loss: In essence this is the min-max loss function. The generator tries to maximise, while the discriminator tries to minimise. 

### The SRGAN Loss function 

The paper proposed a perceptual loss function, which consists of an adversarial loss and a content loss. The content loss is useful for comparing the features of the reconstructed images with the original images, focusing on differences in features rather than differences in pixel levels in attributes like colour and brightness. <br>

In addition, the SRGAN model uses the VGG19 network to extract image features for content loss calculation. The VGG19 convolutional neural network was developed by Karen Simoyan and Andrew Zisserman at the University of Oxford in 2014. The network contains 19 trainable networks, out of which, 16 of those layers are convolutional. The network relies on 3x3 convolutions with a stride of 1(Stride refers to the number of pixels by which we move the filter/kernel across the input image).<br>

By utilising  the VGG19 network, the SRGAN model can extract complex features from images and compare them to enhance image quality. Additionally, the VGG loss helps in preserving high frequency information and fine details in the reconstructed images. The adversarial loss function in the SRGAN model also encourages the generator to produce images that closely resemble high resolution images, further enhancing the overall performance of the model. <br>

As stated earlier, the perceptual loss function consists of an adverserial loss and a content loss. The equation for them is given below. 

$$ \large l^{\text{SR}} = \overbrace{\underbrace{l_x^{\text{SR}}}_{\text{Content Loss}} + \underbrace{10^{-3}l_{\text{Gen}}^{\text{SR}}}_{\text{Adversarial Loss}}}^{\text{Perceptual Loss}} $$

As we can see from the equation, the content loss plays a major role in the loss function of the SRGAN loss function, while the adversarial loss function only has $\frac{1}{1000}$th of weightage in the loss function. <br>

The content loss is composed of 2 parts, MSE Loss, and VGGLoss. The MSE Loss is the pixel wise error between the HR image and the SR image, whereas the VGG loss indicates the feature map obtained by the j-th convolution(post activation) before the i-th maxpooling layer within the VGG19 network. This is also defined as the euclidean distance between the feature representations of a reconstructed image and the reference image. 

The `TVLoss` Class defined below calculates Total Variable Loss, which measures the smoothness of a tensor, by encouranging small differences between neighbouring pixels. Horizontal and vertical variations are also considered, and a single loss value is returned. 

In [ ]:
class TVLoss(nn.Module):
    def __init__(self, tv_loss_weight=1):
        super(TVLoss, self).__init__()
        self.tv_loss_weight=tv_loss_weight
  
    def forward(self, x):
        batch_size=x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]

        count_h = self.tensor_size(x[:, :, 1:, :])
        count_w = self.tensor_size(x[:, :, :, 1:])

        h_tv = torch.pow(x[:, :, 1:, :] - x[:, :, :h_x - 1, :], 2).sum()
        w_tv = torch.pow(x[:, :, :, 1:] - x[:, :, :, :w_x - 1], 2).sum()
        return self.tv_loss_weight * 2 * (h_tv / count_h + w_tv / count_w) / batch_size

    @staticmethod # Must add this
    def tensor_size(t):
        return t.size()[1] * t.size()[2] * t.size()[3]

The `GeneratorLoss` Class is where we have initialised the loss function of the model. This contains of various parts: 
- `vgg`: The pretrained VGG-16 network, which is a CNN used for image recognition. The `pretrained=True` arguments means that the weights of the network are initialised with the weights trained on the ImageNet dataset. 
- `loss_network`: This is a subnetwork of the VGG network, containing the first 31 layers of the network. This subnet is used to calculate the perception loss. 
- `mse_loss`: This is the MSE loss function, used to compare generated images with target images. 
- `tv_loss`: This is the total variation loss used to smooth out the generated images

In [ ]:
class GeneratorLoss(nn.Module):
    def __init__(self):
        super(GeneratorLoss, self).__init__()
        vgg = vgg16(pretrained=True)
        loss_network = nn.Sequential(*list(vgg.features)[:31]).eval()
        for param in loss_network.parameters():
          param.requires_grad = False
        self.loss_network = loss_network
        self.mse_loss = nn.MSELoss()
        self.tv_loss = TVLoss()
        
    def forward(self, out_labels, out_images, target_images):
        adversial_loss = torch.mean(1 - out_labels)
        perception_loss = self.mse_loss(out_images, target_images)
        image_loss = self.mse_loss(out_images, target_images)
        tv_loss = self.tv_loss(out_images)
        return image_loss + 0.001 * adversial_loss + 0.006 * perception_loss + 2e-8 * tv_loss

### Device Selection

This next code cell initialises the device that will be used for training. Since this is trained on a local machine, the model will choose a CUDA capable device(in this case, a Mobile NVIDIA RTX 3070Ti GPU with 8GB of VRAM). 

In [24]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    getGPU_details(device)

GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU
Total Memory: 8.0 GB
Memory Usage:
Allocated: 0.0 GB
Cached: 0.0 GB


In [ ]:
netG = Generator(UPSCALE_FACTOR)
netD = Discriminator()

In [ ]:
generator_criterion = GeneratorLoss()

In [ ]:
generator_criterion = generator_criterion.to(device)
netG = netG.to(device)
netD = netD.to(device)

In [ ]:
optimizerG = optim.Adam(netG.parameters(), lr=0.0002)
optimizerD = optim.Adam(netD.parameters(), lr=0.0002)

In [ ]:
results = {
    "d_loss":[],
    "g_loss":[],
    "d_score": [],
    "g_score": []
}

In [ ]:
N_EPOCHS = 10

In [ ]:
def saveCheckpoint(model, optimizer, filename):
    checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    torch.save(checkpoint,filename)
    print(f'Saved checkpoint {filename}')

In [ ]:
def loadCheckpoint(checkpoint_file, model, optimizer, lr): 
    print('===> load checkpoint')
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    for param_group in optimizer.param_groups:      # needed or model will have learning rate of older checkpoint 
        param_group['lr'] = lr

The Code block below implements the training of the SRGAN model. The discriminator network is updated first, followed by the generator network. The process is repeated for each batch of data until the end of the epoch is reached. The running totals of the losses and scores are used to compute the average loss and score for each epoch. 

In [ ]:
t0 = time()
for epoch in range(1, N_EPOCHS + 1):
    train_bar = tqdm(trainloader, file=sys.stdout)
    running_results = {'batch_sizes':0, 'd_loss':0,
                     "g_loss":0, "d_score":0, "g_score":0}
    netG.train()
    netD.train()
    for data, target in train_bar:
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        real_img = Variable(target)
        real_img = real_img.to(device)
        z = Variable(data)
        z = z.to(device)

        ## Update Discriminator ##
        fake_img = netG(z)
        netD.zero_grad()
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        d_loss = 1 - real_out + fake_out
        d_loss.backward(retain_graph = True)
        optimizerD.step()

        ## Now update Generator
        fake_img = netG(z)
        fake_out = netD(fake_img).mean()
        netG.zero_grad()
        g_loss = generator_criterion(fake_out, fake_img, real_img)
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += real_out.item() * batch_size

        ## Updating the progress bar
        train_bar.set_description(desc="[%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f" % (
            epoch, N_EPOCHS, running_results['d_loss'] / running_results['batch_sizes'],
            running_results['g_loss'] / running_results['batch_sizes'],
            running_results['d_score'] / running_results['batch_sizes'],
            running_results['g_score'] / running_results['batch_sizes']
        ))
    results['d_loss'].append(running_results['d_loss'])
    results['g_loss'].append(running_results['g_loss'])
    results['d_score'].append(running_results['d_score'])
    results['g_score'].append(running_results['g_score'])
    
    if epoch%50==0:
        saveCheckpoint(netG, optimizerG, filename='checkpoints/generator_checkpoint.pth')
        print(f'Saved Generator Checkpoint')
        saveCheckpoint(netD, optimizerD, filename='checkpoints/discriminator_checkpoint.pth')
        print(f'Saved Discriminator Checkpoint')
    netG.eval()
print(f'Total Training Time: {(time() - t0)/3600} Hours')

# An attempt at testing the model

In [ ]:
def testModel(input_file, save_image=False):
    '''
    This function passes an image into the model, and retrieves its input. 
    Need to implement model saving and checkpointing. 
    '''
    # Read input image and convert to Tensor for feeding into model
    image = cv2.imread(input_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Need to add this as OpenCV uses BGR instead of RGB
    transform = transforms.Compose([transforms.ToTensor()])
    in_tensor = transform(image)
    
    # Feed input tensor into model
    in_tensor = in_tensor.to(device) # Send input to device, can cause problems if input tensor is on CPU and Model on GPU!!
    out_tensor = netG(in_tensor.unsqueeze(0))
    
    # Final transformations, as for some reason, output is rotated
    display_tensor = out_tensor.to('cpu') # send output tensor back to CPU
    display_tensor = display_tensor.detach().numpy()
    display_tensor = np.transpose(display_tensor)
    display_tensor = np.squeeze(display_tensor)
    display_tensor = cv2.flip(display_tensor, 0)
    display_tensor = rotate(display_tensor, -90)
    f = (display_tensor * 255).astype(np.uint8)
    if save_image:  
        out_file = input('What name do you want to save the image as: ')
        cv2.imwrite(f'{out_file}.png', f)
    
    return f

In [ ]:
i = readRGBImage('Alsat-2B/Test/Agriculture/HR/HR_12.jpg')
j = readRGBImage('Alsat-2B/Test/Agriculture/LR/LR_12.jpg')
a = testModel('Alsat-2B/Test/Agriculture/LR/LR_12.jpg')
DisplayImageComparison1x3(i, j, a, metrics=True)

In [ ]:
i = readRGBImage('Alsat-2B/Test/Agriculture/HR/HR_51.jpg')
j = readRGBImage('Alsat-2B/Test/Agriculture/LR/LR_51.jpg')
a = testModel('Alsat-2B/Test/Agriculture/LR/LR_51.jpg')
DisplayImageComparison1x3(i, j, a, metrics=True)

In [ ]:
i = readRGBImage('Alsat-2B/Test/Agriculture/HR/HR_11.jpg')
j = readRGBImage('Alsat-2B/Test/Agriculture/LR/LR_11.jpg')
a = testModel('Alsat-2B/Test/Agriculture/LR/LR_11.jpg')
DisplayImageComparison1x3(i, j, a, metrics=True)

In [ ]:
plt.plot([i/100 for i in results['d_loss']], label='Discriminator Loss')
plt.plot([i/100 for i in results['g_loss']], label='Generator Loss')
plt.legend()
plt.xlabel('Epochs')
plt.title('Discriminator vs Generator Loss')

## References: 

1.  Ledig, C., Theis, L., Huszár, F., Caballero, J., Cunningham, A., Acosta, A., ... & Shi, W. (2017). Photo-realistic single image super-resolution using a generative adversarial network. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 4681-4690).
2. Djerida, A., Djerriri, K., & Karoui, M. S. (2021, July). A new public Alsat-2B dataset for single-image super-resolution. In 2021 IEEE International Geoscience and Remote Sensing Symposium IGARSS (pp. 8095-8098). IEEE.
3. Shi, W., Caballero, J., Huszár, F., Totz, J., Aitken, A. P., Bishop, R., ... & Wang, Z. (2016). Real-time single image and video super-resolution using an efficient sub-pixel convolutional neural network. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1874-1883).